# Some Porsche Taycan Statistics


In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
dir = os.getcwd()
dir = r"C:\Users\fbeth\OneDrive\Dokumente"
for file in os.listdir(dir):
    if file.endswith('2.json') and file.__contains__('short'):
        file = os.path.join(dir, file)
        break

print (f"reading file {file}")
df = pd.read_json(file)
df.info()


__________
### clean some columns 

also for nicer usage

In [ ]:
def clean_data(df):
     
    del df[df.averageFuelConsumption.name] #not needed on an electric car

    df['km']=[d.get('value') for d in df.zeroEmissionDistance]
    del df[df.zeroEmissionDistance.name]
    df['km2']=[d.get('value') for d in df.tripMileage]
    del df[df.tripMileage.name]
    #
    df['consump'] = df.averageElectricEngineConsumption.apply(lambda x: x.get('value'))
    df.drop(df.averageElectricEngineConsumption.name, axis=1, inplace=True)
    #
    df['avgSpeed'] = df.averageSpeed.apply(lambda x: x.get('value'))
    del df[df.averageSpeed.name]
    # odo
    df['odo'] = df.endMileage.apply(lambda x: x.get('value'))
    del df[df.endMileage.name]
    #
    del df[df.startMileage.name]

    return df


df = clean_data(df)




### drop unplausible or unwanted periods if any


In [ ]:
df= df.set_index('timestamp').loc['2020-10-18':] #'2022-09-19']
df.index

In [ ]:
df.query('km >90') # shwo trips longer than 90 km


In [ ]:
#df.query('km > 10',inplace=True) # reduce noise in the data due to short trips = cold starts.

df['rol'] = df.consump.rolling(20).mean()

df.plot(kind="line",y=['rol','consump'],figsize=(12,8),title='Consumption',grid=True)
#x2.plot(kind='bar', figsize=(12,6))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(df.rol["2022"], color = (0,0,0), linewidth = 4, alpha = .9, label = 'Smoothed');
ax.plot(df.consump["2022"], color = (1,0,1), label = 'Original');
ax.set_title('Taycan consumption')
ax.set_xlabel('Time')
ax.set_ylabel('consump')
ax.legend(loc='upper right');

In [ ]:
plt.xlabel('kwh/100km')
plt.ylabel('trips > 10km')
plt.title('Histogram of consumption')
plt.ylim(0,10)
plt.xlim(10,40)
plt.hist(df.consump["2022"].where(df.km > 10), bins = np.arange(10,40,0.5) )
plt.show()